In [23]:
import torch
from torch.autograd import grad
import torch.nn as nn
import numpy as np
import math
import torch.optim as optim

import h5py
import scipy.io

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.set_default_dtype(torch.float32)

In [24]:
f = 0.06
e = 0.045
l_1 = 0.176
g = 9.8
alpha = torch.tensor([-30, 90, 210], dtype = torch.float32)

In [25]:
class PINN_only_Friction(nn.Module):
    
    #----------------------------Initialize the Network--------------------------#
    def __init__(self, input_dim = 10, output_dim = 6, hidden_layer_dim = 64, num_hidden_layer = 3, activation = 'tanh'):
        super(PINN_only_Friction, self).__init__()
        
        #----------------------Initializing Parameters----------------------------#
        self.l_1c = 85 * (10 ** -3)
        self.m_0 = 14 * (10 ** -3)
        self.m_1 = 22.80 * (10 ** -3)
        self.m_2 = 150 * (10 ** -3)
        self.I_1 = 256400.64 * (10 ** -9) + self.m_1 * l_1 ** 2

        self.layers = nn.ModuleList([nn.Linear(input_dim, hidden_layer_dim)])
        for _ in range(num_hidden_layer - 1):
            self.layers.append(nn.Linear(hidden_layer_dim, hidden_layer_dim))
        self.layers.append(nn.Linear(hidden_layer_dim, output_dim))

        self.epoch = 0
        
        
        #---------------------Differences compare to Comprehensive model-------------------#
        self.f_c1 = nn.Parameter(torch.tensor(np.random.rand()), requires_grad = True)
        self.f_c2 = nn.Parameter(torch.tensor(np.random.rand()), requires_grad = True)
        self.f_c3 = nn.Parameter(torch.tensor(np.random.rand()), requires_grad = True)

        self.f_v1 = nn.Parameter(torch.tensor(np.random.rand()), requires_grad = True)
        self.f_v2 = nn.Parameter(torch.tensor(np.random.rand()), requires_grad = True)
        self.f_v3 = nn.Parameter(torch.tensor(np.random.rand()), requires_grad = True)


        if activation == 'sin':
          self.activation = torch.sin
        elif activation == 'tanh':
          self.activation = torch.tanh
        elif activation == 'relu':
          self.activation = torch.nn.ReLU()
        else:
          self.activation = torch.nn.LeakyReLU()
          
    #------------------------Forward Pass---------------------------#      
    def forward(self, t, tau, s, s_Ddot):
        out = torch.cat([t, tau, s, s_Ddot], dim=-1)
        for layer in self.layers[:-1]:
            out = self.activation(layer(out))
        out = self.layers[-1](out)
        return out
      
    #------------------------Define Inverse Dynamic equations-----------------#
    def Inverse_Dynamic_Eq(self, t, tau, s, s_Ddot):
        f_c1 = torch.nn.functional.softplus(self.f_c1)
        f_c2 = torch.nn.functional.softplus(self.f_c2)
        f_c3 = torch.nn.functional.softplus(self.f_c3)
        f_v1 = torch.nn.functional.softplus(self.f_v1)
        f_v2 = torch.nn.functional.softplus(self.f_v2)
        f_v3 = torch.nn.functional.softplus(self.f_v3)
      
        t = t.requires_grad_()
        u_pred = self.forward(t, tau, s, s_Ddot)   # batch_size x 6

        theta, theta_dot = torch.chunk(u_pred, 2, dim = 1) # each has dimension of: batch_size x 3

        theta_1 = theta[:, 0:1]                     # shape: batch_size x 1
        theta_2 = theta[:, 1:2]                     # shape: batch_size x 1
        theta_3 = theta[:, 2:3]                     # shape: batch_size x 1

        theta_1_dot = theta_dot[:, 0:1]             # shape: batch_size x 1
        theta_2_dot = theta_dot[:, 1:2]             # shape: batch_size x 1
        theta_3_dot = theta_dot[:, 2:3]             # shape: batch_size x 1

        
        #---------------Define M matrix------------------------#
        M = torch.cat(                                                                                                  # 6 x 6
                    [torch.cat([torch.eye(3), torch.zeros(3, 3)], dim=1),                                               # 3 x 6
                     torch.cat([torch.zeros(3, 3), (2 * self.I_1 + self.m_2 * (l_1 ** 2)) * torch.eye(3)], dim=1)],     # 3 x 6
                     dim=0
                     ).unsqueeze(0).expand(u_pred.shape[0], -1, -1)
        
        #M = M.to(device)


        #---------------Define G matrix------------------------#
        G_1 = (self.m_1 * self.l_1c + self.m_2 * l_1) * g * torch.cos(torch.deg2rad(theta_1))               # batch_size x 1
        G_2 = (self.m_1 * self.l_1c + self.m_2 * l_1) * g * torch.cos(torch.deg2rad(theta_2))               # batch_size x 1
        G_3 = (self.m_1 * self.l_1c + self.m_2 * l_1) * g * torch.cos(torch.deg2rad(theta_3))               # batch_size x 1

        G = torch.cat([G_1, G_2, G_3], dim = 1)                                                             # batch_size x 3
        #G = G.to(device)

        #----------------Define Friction matrices---------------#
        F_v = torch.zeros(3, 3)
        F_v[0, 0] = self.f_v1
        F_v[1, 1] = self.f_v2
        F_v[2, 2] = self.f_v3
        F_v = F_v.unsqueeze(0).expand(s.shape[0],-1, -1)
        
        F_c = torch.zeros(3, 3)
        F_c[0, 0] = self.f_c1
        F_c[1, 1] = self.f_c2
        F_c[2, 2] = self.f_c3
        F_c = F_c.unsqueeze(0).expand(s.shape[0],-1, -1)
        
        #F_v = F_v.to(device)
        #F_c = F_c.to(device)

        #---------------Computing derivatives----------------#
        theta_1_dot_t = torch.autograd.grad(theta_1, t, grad_outputs = torch.ones_like(theta_1), create_graph=True)[0]
        theta_2_dot_t = torch.autograd.grad(theta_2, t, grad_outputs = torch.ones_like(theta_2), create_graph=True)[0]
        theta_3_dot_t = torch.autograd.grad(theta_3, t, grad_outputs = torch.ones_like(theta_3), create_graph=True)[0]

        theta_1_Ddot_t = torch.autograd.grad(theta_1_dot, t, grad_outputs = torch.ones_like(theta_1_dot), create_graph=True)[0]
        theta_2_Ddot_t = torch.autograd.grad(theta_2_dot, t, grad_outputs = torch.ones_like(theta_2_dot), create_graph=True)[0]
        theta_3_Ddot_t = torch.autograd.grad(theta_3_dot, t, grad_outputs = torch.ones_like(theta_3_dot), create_graph=True)[0]


        #----------------Define du_t = [theta_1_dot theta_2_dot theta_3_dot theta_1_Ddot theta_2_Ddot theta_3_Ddot]--------------#
        du_t = torch.cat([theta_1_dot_t, theta_2_dot_t, theta_3_dot_t, theta_1_Ddot_t, theta_2_Ddot_t, theta_3_Ddot_t], dim = 1).unsqueeze(-1)   # batch_size x 6 x 1

        #du_t = du_t.to(device)
        
        #----------------Define K matrix---------------------#
        K11 = (s[:, 0] * torch.cos(torch.deg2rad(alpha[0])) + s[:, 1] * torch.sin(torch.deg2rad(alpha[0])) + f - e) * torch.sin(torch.deg2rad(theta[:, 0])) - s[:, 2] * torch.cos(torch.deg2rad(theta[:, 0]))
        K22 = (s[:, 0] * torch.cos(torch.deg2rad(alpha[1])) + s[:, 1] * torch.sin(torch.deg2rad(alpha[1])) + f - e) * torch.sin(torch.deg2rad(theta[:, 1])) - s[:, 2] * torch.cos(torch.deg2rad(theta[:, 1]))
        K33 = (s[:, 0] * torch.cos(torch.deg2rad(alpha[2])) + s[:, 1] * torch.sin(torch.deg2rad(alpha[2])) + f - e) * torch.sin(torch.deg2rad(theta[:, 2])) - s[:, 2] * torch.cos(torch.deg2rad(theta[:, 2]))

        K = torch.zeros((s.shape[0], 3, 3))    # batch_size x 3 x 3
        K[:, 0, 0] = K11
        K[:, 1, 1] = K22
        K[:, 2, 2] = K33

        #----------------------------------------------------#
        a_11 = s[:, 0] + e * torch.cos(torch.deg2rad(alpha[0])) - f * torch.cos(torch.deg2rad(alpha[0])) - l_1 * torch.cos(torch.deg2rad(alpha[0])) * torch.cos(torch.deg2rad(theta[:, 0])) # shape: batch_size
        a_12 = s[:, 0] + e * torch.cos(torch.deg2rad(alpha[1])) - f * torch.cos(torch.deg2rad(alpha[1])) - l_1 * torch.cos(torch.deg2rad(alpha[1])) * torch.cos(torch.deg2rad(theta[:, 1])) # shape: batch_size
        a_13 = s[:, 0] + e * torch.cos(torch.deg2rad(alpha[2])) - f * torch.cos(torch.deg2rad(alpha[2])) - l_1 * torch.cos(torch.deg2rad(alpha[2])) * torch.cos(torch.deg2rad(theta[:, 2])) # shape: batch_size

        a_21 = s[:, 1] + e * torch.sin(torch.deg2rad(alpha[0])) - f * torch.sin(torch.deg2rad(alpha[0])) - l_1 * torch.sin(torch.deg2rad(alpha[0])) * torch.cos(torch.deg2rad(theta[:, 0])) # shape: batch_size
        a_22 = s[:, 1] + e * torch.sin(torch.deg2rad(alpha[1])) - f * torch.sin(torch.deg2rad(alpha[1])) - l_1 * torch.sin(torch.deg2rad(alpha[1])) * torch.cos(torch.deg2rad(theta[:, 1])) # shape: batch_size
        a_23 = s[:, 1] + e * torch.sin(torch.deg2rad(alpha[2])) - f * torch.sin(torch.deg2rad(alpha[2])) - l_1 * torch.sin(torch.deg2rad(alpha[2])) * torch.cos(torch.deg2rad(theta[:, 2])) # shape: batch_size

        a_31 = s[:, 2] - l_1 * torch.cos(torch.deg2rad(theta[:, 0])) # shape: batch_size
        a_32 = s[:, 2] - l_1 * torch.cos(torch.deg2rad(theta[:, 1])) # shape: batch_size
        a_33 = s[:, 2] - l_1 * torch.cos(torch.deg2rad(theta[:, 2])) # shape: batch_size

        #-------------Define A matrix--------#
        A = torch.zeros((s.shape[0], 3, 3))

        A[:, 0, 0] = a_11
        A[:, 0, 1] = a_12
        A[:, 0, 2] = a_13
        A[:, 1, 0] = a_21
        A[:, 1, 1] = a_22
        A[:, 1, 2] = a_23
        A[:, 2, 0] = a_31
        A[:, 2, 1] = a_32
        A[:, 2, 2] = a_33
        
        #------------Define B matrix-----------#
        B = torch.zeros((s.shape[0], 3, 1))
        
        B[:, 0, 0] = (self.m_0 + 3 * self.m_2) * s_Ddot[:, 0]
        B[:, 1, 0] = (self.m_0 + 3 * self.m_2) * s_Ddot[:, 1]
        B[:, 2, 0] = (self.m_0 + 3 * self.m_2) * (s_Ddot[:, 2] - g)

        #equation = ((M @ du_t) -                                                                                                                                                # [batch_size x 6 x 6 @ batch_size x 6 x 1] = batch_size x 6 x 1
                    #(torch.cat((theta_dot.unsqueeze(-1), - G.unsqueeze(-1) + K @ torch.linalg.inv(A) @ B - (F_v @ theta_dot.unsqueeze(-1)) - ((F_c @ torch.sign(theta_dot.unsqueeze(-1))))), dim = 1)) -    # batch_size x 3 -> unsqueeze(-1) = batch_size x 3 x 1
                                                                                                                                                                                                  # batch_size x 3 x 3 @ batch_size x 3 x 3 @ batch_size x 3 x 1 - [3 x 3 @ (batch_size x 3)T]T - [3 x 3 @ (batch_size x 3)T)]T = batch_size x 3 x 1
                                                                                                                                                                                                  #                                                                       batch_size x 3 -> unsqueeze(-1) = batch_size x 3 x 1
                                                                                                                                                                                                  # Attention: dim = 1 to concatenate along rows (1st dimension)
                    #torch.cat([torch.zeros_like(tau), tau], dim = 1).unsqueeze(-1))     #   batch_size x 3
                                                                                        #   batch_size x 3
                                                                                        # = batch_size x 6 -> unsqueeze(-1) = batch_size x 6 x 1

        equation = torch.cat((theta_dot.unsqueeze(-1), - G.unsqueeze(-1) + K @ torch.linalg.inv(A) @ B - (F_v @ theta_dot.unsqueeze(-1)) - ((F_c @ torch.sign(theta_dot.unsqueeze(-1))))), dim = 1)
        equation = equation.squeeze(-1)

        return equation
    
    #----------------------------------PDE Residual Loss function---------------------------------------#
    def pde_loss(self, t, tau, s, s_Ddot):      
        
        loss_r1 = torch.mean(self.Inverse_Dynamic_Eq(t, tau, s, s_Ddot)[:, 3:4] ** 2)
        loss_r2 = torch.mean(self.Inverse_Dynamic_Eq(t, tau, s, s_Ddot)[:, 4:5] ** 2) 
        loss_r3 = torch.mean(self.Inverse_Dynamic_Eq(t, tau, s, s_Ddot)[:, 5:6] ** 2) 
        return loss_r1, loss_r2, loss_r3
    
    #----------------------------------Data Loss function-----------------------------------------------#
    def data_loss(self, t, tau, s, s_Ddot, u_data):   
                                                                                                        # The first dimension across all argument is batch_size
        pred = self.forward(t, tau, s, s_Ddot)                                                          # In order to compute the mean of pass-in values, we do not need to preserve the dimensions of tensor
        loss_theta_1 = torch.mean((pred[:, 0:1] - u_data[:, 0:1]) ** 2)                                 # So that, we use [:, i] not [:, i:i+1]
        loss_theta_2 = torch.mean((pred[:, 1:2] - u_data[:, 1:2]) ** 2) 
        loss_theta_3 = torch.mean((pred[:, 2:3] - u_data[:, 2:3]) ** 2)
        loss_theta_1_dot = torch.mean((pred[:, 3:4] - u_data[:, 3:4]) ** 2) 
        loss_theta_2_dot = torch.mean((pred[:, 4:5] - u_data[:, 4:5]) ** 2) 
        loss_theta_3_dot = torch.mean((pred[:, 5:6] - u_data[:, 5:6]) ** 2) 
        return loss_theta_1, loss_theta_2, loss_theta_3, loss_theta_1_dot, loss_theta_2_dot, loss_theta_3_dot

In [26]:
#----------------------------------Adaptive Weight function-----------------------------------------#
#---------------------------------Execute in each epoch---------------------------------------------#
def adaptive_weight(model, X_train):
    model.zero_grad()
    param_tensors = [param for param in model.parameters()]
    param_tensors_cpu = [param.cpu() for param in model.parameters()]

    t = X_train['PDE']['t'].requires_grad_()     # total_data x 1
    tau = X_train['PDE']['tau']                  # total_data x 3
    s = X_train['PDE']['s']                      # total_data x 3
    s_Ddot = X_train['PDE']['s_Ddot']            # total_data x 3


    batch_size = 64
    total_data = t.shape[0]
    batches = (total_data + batch_size - 1) // batch_size

    jacobian_r1 = torch.zeros(total_data, sum(p.numel() for p in param_tensors_cpu))
    jacobian_r2 = torch.zeros(total_data, sum(p.numel() for p in param_tensors_cpu))
    jacobian_r3 = torch.zeros(total_data, sum(p.numel() for p in param_tensors_cpu))

    for batch in range(batches):
      start_idx = batch * batch_size
      end_idx = min(start_idx + batch_size, total_data)

      t_batch = t[start_idx:end_idx]              # batch_size x 1
      tau_batch = tau[start_idx:end_idx]          # batch_size x 3
      s_batch = s[start_idx:end_idx]              # batch_size x 3
      s_Ddot_batch = s_Ddot[start_idx:end_idx]    # batch_size x 3

      equation = model.Inverse_Dynamic_Eq(t_batch, tau_batch, s_batch, s_Ddot_batch)          # batch_size x 6 x 1

      for i in range(equation.shape[0]):
        grad_output = torch.zeros_like(equation[:, 3:4])
        grad_output[i] = 1

        grads = [g.cpu() if g is not None else None for g in torch.autograd.grad(equation[:, 3:4], param_tensors, grad_output, allow_unused=True, create_graph=True)]
        grad_row = torch.cat([g.view(-1) if g is not None else torch.zeros(p.numel()) for g, p in zip(grads, param_tensors_cpu)])
        jacobian_r1[start_idx + i] = grad_row

        grads = [g.cpu() if g is not None else None for g in torch.autograd.grad(equation[:, 4:5], param_tensors, grad_output, allow_unused=True, create_graph=True)]
        grad_row = torch.cat([g.view(-1) if g is not None else torch.zeros(p.numel()) for g, p in zip(grads, param_tensors_cpu)])
        jacobian_r2[start_idx + i] = grad_row

        grads = [g.cpu() if g is not None else None for g in torch.autograd.grad(equation[:, 5:6], param_tensors, grad_output, allow_unused=True, create_graph=True)]
        grad_row = torch.cat([g.view(-1) if g is not None else torch.zeros(p.numel()) for g, p in zip(grads, param_tensors_cpu)])
        jacobian_r3[start_idx + i] = grad_row


    t = X_train['data']['t'].requires_grad_()
    tau = X_train['data']['tau']
    s = X_train['data']['s']
    s_Ddot = X_train['data']['s_Ddot']


    batch_size = 32
    total_data = t.shape[0]
    batches = (total_data + batch_size - 1) // batch_size

    jacobian_theta_1 = torch.zeros(total_data, sum(p.numel() for p in param_tensors_cpu))
    jacobian_theta_2 = torch.zeros(total_data, sum(p.numel() for p in param_tensors_cpu))
    jacobian_theta_3 = torch.zeros(total_data, sum(p.numel() for p in param_tensors_cpu))
    jacobian_theta_1_dot = torch.zeros(total_data, sum(p.numel() for p in param_tensors_cpu))
    jacobian_theta_2_dot = torch.zeros(total_data, sum(p.numel() for p in param_tensors_cpu))
    jacobian_theta_3_dot = torch.zeros(total_data, sum(p.numel() for p in param_tensors_cpu))

    for batch in range(batches):
      start_idx = batch * batch_size
      end_idx = min(start_idx + batch_size, total_data)

      t_batch = t[start_idx:end_idx]
      tau_batch = tau[start_idx:end_idx]
      s_batch = s[start_idx:end_idx]
      s_Ddot_batch = s_Ddot[start_idx:end_idx]

      u_pred = model.forward(t_batch, tau_batch, s_batch, s_Ddot_batch)         # batch_size x 6

      for i in range(len(u_pred)):
        grad_output = torch.zeros(u_pred[:, 0].shape)
        grad_output[i] = 1

        grads = [g.cpu() if g is not None else None for g in torch.autograd.grad(u_pred[:, 0], param_tensors, grad_output, allow_unused=True, create_graph=True)]
        grad_row = torch.cat([g.view(-1) if g is not None else torch.zeros(p.numel()) for g, p in zip(grads, param_tensors_cpu)])
        jacobian_theta_1[start_idx + i] = grad_row

        grads = [g.cpu() if g is not None else None for g in torch.autograd.grad(u_pred[:, 1], param_tensors, grad_output, allow_unused=True, create_graph=True)]
        grad_row = torch.cat([g.view(-1) if g is not None else torch.zeros(p.numel()) for g, p in zip(grads, param_tensors_cpu)])
        jacobian_theta_2[start_idx + i] = grad_row

        grads = [g.cpu() if g is not None else None for g in torch.autograd.grad(u_pred[:, 2], param_tensors, grad_output, allow_unused=True, create_graph=True)]
        grad_row = torch.cat([g.view(-1) if g is not None else torch.zeros(p.numel()) for g, p in zip(grads, param_tensors_cpu)])
        jacobian_theta_3[start_idx + i] = grad_row

        grads = [g.cpu() if g is not None else None for g in torch.autograd.grad(u_pred[:, 3], param_tensors, grad_output, allow_unused=True, create_graph=True)]
        grad_row = torch.cat([g.view(-1) if g is not None else torch.zeros(p.numel()) for g, p in zip(grads, param_tensors_cpu)])
        jacobian_theta_1_dot[start_idx + i] = grad_row

        grads = [g.cpu() if g is not None else None for g in torch.autograd.grad(u_pred[:, 4], param_tensors, grad_output, allow_unused=True, create_graph=True)]
        grad_row = torch.cat([g.view(-1) if g is not None else torch.zeros(p.numel()) for g, p in zip(grads, param_tensors_cpu)])
        jacobian_theta_2_dot[start_idx + i] = grad_row

        grads = [g.cpu() if g is not None else None for g in torch.autograd.grad(u_pred[:, 5], param_tensors, grad_output, allow_unused=True, create_graph=True)]
        grad_row = torch.cat([g.view(-1) if g is not None else torch.zeros(p.numel()) for g, p in zip(grads, param_tensors_cpu)])
        jacobian_theta_3_dot[start_idx + i] = grad_row

      Krr11 = jacobian_r1 @ jacobian_r1.T
      Krr22 = jacobian_r2 @ jacobian_r2.T
      Krr33 = jacobian_r3 @ jacobian_r3.T
      Kuu11 = jacobian_theta_1 @ jacobian_theta_1.T
      Kuu22 = jacobian_theta_2 @ jacobian_theta_2.T
      Kuu33 = jacobian_theta_3 @ jacobian_theta_3.T
      Kuu44 = jacobian_theta_1_dot @ jacobian_theta_1_dot.T
      Kuu55 = jacobian_theta_2_dot @ jacobian_theta_2_dot.T
      Kuu66 = jacobian_theta_3_dot @ jacobian_theta_3_dot.T

      K_trace = (torch.trace(Krr11) + torch.trace(Krr22) + torch.trace(Krr33)) / batch_sizes['PDE'] + \
                (torch.trace(Kuu11) + torch.trace(Kuu22) + torch.trace(Kuu33) + torch.trace(Kuu44) + torch.trace(Kuu55) + torch.trace(Kuu66))/batch_sizes['data']

      lambda_1 = K_trace / (torch.trace(Krr11)/batch_sizes['PDE'])
      lambda_2 = K_trace / (torch.trace(Krr22)/batch_sizes['PDE'])
      lambda_3 = K_trace / (torch.trace(Krr33)/batch_sizes['PDE'])
      lambda_4 = K_trace / (torch.trace(Kuu11)/batch_sizes['data'])
      lambda_5 = K_trace / (torch.trace(Kuu22)/batch_sizes['data'])
      lambda_6 = K_trace / (torch.trace(Kuu33)/batch_sizes['data'])
      lambda_7 = K_trace / (torch.trace(Kuu44)/batch_sizes['data'])
      lambda_8 = K_trace / (torch.trace(Kuu55)/batch_sizes['data'])
      lambda_9 = K_trace / (torch.trace(Kuu66)/batch_sizes['data'])

      lambda1 = (lambda_1.item())
      lambda2 = (lambda_2.item())
      lambda3 = (lambda_3.item())
      lambda4 = (lambda_4.item())
      lambda5 = (lambda_5.item())
      lambda5 = (lambda_5.item())
      lambda6 = (lambda_6.item())
      lambda7 = (lambda_7.item())
      lambda8 = (lambda_8.item())
      lambda9 = (lambda_9.item())

      return lambda1, lambda2, lambda3, lambda4, lambda5, lambda6, lambda7, lambda8, lambda9


In [27]:

#---------------------------------Getting Data from .mat files-----------------------------#
def get_Data(batch_sizes):
    x = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_x.mat", "r")["data_x"][3:49998, 1]).unsqueeze(1)                # 1996 x 1
    y = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_y.mat", "r")["data_y"][3:49998, 1]).unsqueeze(1)                # 1996 x 1
    z = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_z.mat", "r")["data_z"][3:49998, 1]).unsqueeze(1)                # 1996 x 1

    s = torch.cat((x, y, z), dim=1) * 0.001                                                                       # 1996 x 3

    x_Ddot = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_x_Ddot.mat", "r")["data_x_Ddot"][3:49998, 1]).unsqueeze(1) # 1996 x 1
    y_Ddot = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_y_Ddot.mat", "r")["data_y_Ddot"][3:49998, 1]).unsqueeze(1) # 1996 x 1
    z_Ddot = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_z_Ddot.mat", "r")["data_z_Ddot"][3:49998, 1]).unsqueeze(1) # 1996 x 1

    s_Ddot = torch.cat((x_Ddot, y_Ddot, z_Ddot), dim=1) * 0.001                                                   # 1996 x 3

    t = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_x.mat", "r")["data_x"][3:49998, 0]).unsqueeze(1).float()        # 1996 x 1

    tau_1 = torch.tensor(scipy.io.loadmat("C:/Users/FPTSHOP/Desktop/Project/tau_1.mat")["tau_first_row"][0, 3:49998]).squeeze().unsqueeze(1)        # 1996 x 1
    tau_2 = torch.tensor(scipy.io.loadmat("C:/Users/FPTSHOP/Desktop/Project/tau_1.mat")["tau_second_row"][0, 3:49998]).squeeze().unsqueeze(1)        # 1996 x 1
    tau_3 = torch.tensor(scipy.io.loadmat("C:/Users/FPTSHOP/Desktop/Project/tau_1.mat")["tau_third_row"][0, 3:49998]).squeeze().unsqueeze(1)        # 1996 x 1

    tau = torch.cat((tau_1, tau_2, tau_3), dim = 1)                                                               # 1996 x 3

    theta_1 = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_angle1.mat", "r")["angle1"][3:49998, 1]).unsqueeze(1)     # 1996 x 1
    theta_2 = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_angle2.mat", "r")["angle2"][3:49998, 1]).unsqueeze(1)     # 1996 x 1
    theta_3 = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_angle3.mat", "r")["angle3"][3:49998, 1]).unsqueeze(1)     # 1996 x 1

    theta_1_dot = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_angle1_dot.mat", "r")["angle1_dot"][3:49998, 1]).unsqueeze(1)                # 1996 x 1
    theta_2_dot = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_angle2_dot.mat", "r")["angle2_dot"][3:49998, 1]).unsqueeze(1)                # 1996 x 1
    theta_3_dot = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_angle3_dot.mat", "r")["angle3_dot"][3:49998, 1]).unsqueeze(1)                # 1996 x 1

    u_data = torch.cat((theta_1, theta_2, theta_3, theta_1_dot, theta_2_dot, theta_3_dot), dim = 1)               # 1996 x 6

    data = torch.cat((s, s_Ddot, t, tau), dim = 1)                                                                # 1996 x 10

    total_points = t.shape[0]                                                                                     # 1996
    ids_total = np.arange(total_points)                                                                           # [0, 1, 2, 3, ..., 1994, 1995]

    def random_selection(id_set, size):
        np.random.seed(seeds_num)
        return np.random.choice(id_set, size, replace=False)

    def to_tensor(ids):
        return {
            's'     : data[ids, 0:3].float(),
            's_Ddot': data[ids, 3:6].float(),
            't'     : data[ids, 6:7].float(),
            'tau'   : data[ids, 7:10].float(),
            'u'     : u_data[ids, :].float(),
        }

    id_pde = random_selection(ids_total, batch_sizes['PDE'])
    id_data = random_selection(ids_total, batch_sizes['data'])

    X_train = {'PDE': to_tensor(id_pde), 'data': to_tensor(id_data)}

    all_train_ids = np.union1d(id_pde, id_data)
    id_remaining = np.setdiff1d(np.arange(total_points), all_train_ids)

    X_test = to_tensor(id_remaining)
    X_true = to_tensor(ids_total)

    return X_train, X_test, X_true

In [28]:
#-------------------------Implement 2-stage optimizer with the first half of the training utilizes Adam and L-GFGS for the second half----------------------#
model = PINN_only_Friction()
#dynamic = Dynamic_Model()

all_params = list(model.parameters()) + [model.f_c1, model.f_c2, model.f_c3, model.f_v1, model.f_v2, model.f_v3]

optimizer_Adam = optim.Adam(all_params, lr=0.001, betas=(0.9, 0.999), eps=1e-08)
optimizer_L_BFGS = optim.LBFGS(all_params, lr=0.01, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, history_size=100, line_search_fn=None)


def train(model, optimizer_Adam, optimizer_L_BFGS, X_train, changing_point=1.5, iteration=20000):    
    
    for epoch in range(iteration):

        global lambda_1, lambda_2, lambda_3, lambda_4, lambda_5, lambda_6, lambda_7, lambda_8, lambda_9
    
        loss_r1, loss_r2, loss_r3 = model.pde_loss(X_train['PDE']['t'],
                                                   X_train['PDE']['tau'],
                                                   X_train['PDE']['s'],
                                                   X_train['PDE']['s_Ddot'])
        loss_theta_1, loss_theta_2, loss_theta_3, loss_theta_1_dot, loss_theta_2_dot, loss_theta_3_dot = model.data_loss(
            X_train['data']['t'],
            X_train['data']['tau'],
            X_train['data']['s'],
            X_train['data']['s_Ddot'],
            X_train['data']['u']
        )
        
        
        #------------Calculate the total loss--------------#
        loss = (lambda_1 * loss_r1 + lambda_2 * loss_r2 + lambda_3 * loss_r3 +
                lambda_4 * loss_theta_1 + lambda_5 * loss_theta_2 + lambda_6 * loss_theta_3 +
                lambda_7 * loss_theta_1_dot + lambda_8 * loss_theta_2_dot + lambda_9 * loss_theta_3_dot)


        
        #-----------------------If the current epoch belongs to the first half, use Adam. Otherwise, use L_BGFS-------------------#
        if epoch < changing_point * iteration:
            optimizer_Adam.zero_grad()      # Zero the gradients
            loss.backward()                 # Backpropagate the loss
            optimizer_Adam.step()           # Update parameters with Adam
        else:
            def closure():
                loss_r1, loss_r2, loss_r3 = model.pde_loss(X_train['PDE']['t'],
                                                           X_train['PDE']['tau'],
                                                           X_train['PDE']['s'],
                                                           X_train['PDE']['s_Ddot'])
                
                loss_theta_1, loss_theta_2, loss_theta_3, loss_theta_1_dot, loss_theta_2_dot, loss_theta_3_dot = model.data_loss(
                    X_train['data']['t'],
                    X_train['data']['tau'],
                    X_train['data']['s'],
                    X_train['data']['s_Ddot'],
                    X_train['data']['u']
                )

                #------------Calculate the total loss--------------#
                loss = (loss_r1 + loss_r2 + loss_r3 +
                        loss_theta_1 + loss_theta_2 + loss_theta_3 +
                        loss_theta_1_dot + loss_theta_2_dot + loss_theta_3_dot )
                
                optimizer_L_BFGS.zero_grad()                    # Zero the gradients for L-BFGS
                loss.backward(retain_graph=True)                # Backpropagate the loss
                return loss
            
            #-------------------Use L-BFGS for the second half of the iterations---------------------#
            optimizer_L_BFGS.step(closure)

        model.epoch += 1

        
        #------------------For each 100 epochs, print out the loss value----------------------------#
        if epoch % 100 == 0:
                        
            print(  f"Epoch {epoch:04d}, Loss: {loss.item():.6f}, \n"
                    f"Loss_residual_1: {loss_r1:.6f}, "
                    f"Loss_residual_2: {loss_r2:.6f}, "
                    f"Loss_residual_3: {loss_r3:.6f}, \n"
                    f"Loss_theta_1: {loss_theta_1:.6f}, "
                    f"Loss_theta_2: {loss_theta_2:.6f}, Loss_theta_3: {loss_theta_3:.6f}, \n"
                    f"Loss_theta_1_dot: {loss_theta_1_dot:.6f}, "
                    f"Loss_theta_2_dot: {loss_theta_2_dot:.6f}, "
                    f"Loss_theta_3_dot: {loss_theta_3_dot:.6f}"  )
            
            print(  f"lambda1 {lambda_1: .6f}, "
                    f"lambda2 {lambda_2: .6f}, "
                    f"lambda3 {lambda_3: .6f}, \n"
                    f"lambda4 {lambda_4: .6f}, "
                    f"lambda5 {lambda_5: .6f}, "
                    f"lambda6 {lambda_6: .6f}, \n"
                    f"lambda7 {lambda_7: .6f}, "
                    f"lambda8 {lambda_8: .6f}, "
                    f"lambda9 {lambda_9: .6f}, \n"   )
            
            print(  f"f_v1: {model.f_v1.item(): .6f}, "
                    f"f_v2: {model.f_v2.item(): .6f}, "
                    f"f_v3: {model.f_v3.item(): .6f}, \n"
                    f"f_c1: {model.f_c1.item(): .6f}, "
                    f"f_c2: {model.f_c2.item(): .6f}, "
                    f"f_c3: {model.f_c3.item(): .6f}, \n"
                    "|_________________________________| \n"   )
            
            print("Gradients of f_c and f_v parameters:")
            print(f"f_c1 gradient: {model.f_c1.grad}")
            print(f"f_c2 gradient: {model.f_c2.grad}")
            print(f"f_c3 gradient: {model.f_c3.grad}")
            print(f"f_v1 gradient: {model.f_v1.grad}")
            print(f"f_v2 gradient: {model.f_v2.grad}")
            print(f"f_v3 gradient: {model.f_v3.grad}")
            

            #-----------------------Update adaptive weightings---------------------#
            lambda_1, lambda_2, lambda_3, lambda_4, lambda_5, lambda_6, lambda_7, lambda_8, lambda_9 = adaptive_weight(model, X_train)


In [29]:
seeds_num = 8386
torch.manual_seed(seeds_num)
np.random.seed(seeds_num)
batch_sizes = {'PDE'  : 100,
               'data' : 50}

X_train, X_test, X_true = get_Data(batch_sizes)

lambda_1, lambda_2, lambda_3, lambda_4, lambda_5, lambda_6, lambda_7, lambda_8, lambda_9 = adaptive_weight(model, X_train)

train(model, optimizer_Adam, optimizer_L_BFGS, X_train)


Epoch 0000, Loss: 36325.863281, 
Loss_residual_1: 2.357324, Loss_residual_2: 1.963481, Loss_residual_3: 0.473096, 
Loss_theta_1: 1377.481445, Loss_theta_2: 674.740173, Loss_theta_3: 1039.062622, 
Loss_theta_1_dot: 165.550659, Loss_theta_2_dot: 169.541412, Loss_theta_3_dot: 149.166168
lambda1  8.287048, lambda2  6.476206, lambda3  8.331356, 
lambda4  11.071783, lambda5  9.669416, lambda6  9.371376, 
lambda7  10.060722, lambda8  9.860734, lambda9  9.649278, 

f_v1:  0.851876, f_v2:  0.966759, f_v3:  0.835097, 
f_c1:  0.925343, f_c2:  0.736948, f_c3:  0.278512, 
|_________________________________| 

Gradients of f_c and f_v parameters:
f_c1 gradient: 23.267894744873047
f_c2 gradient: 14.414267539978027
f_c3 gradient: -5.57380485534668
f_v1 gradient: 0.9948859810829163
f_v2 gradient: 0.25627654790878296
f_v3 gradient: -0.09481917321681976
Epoch 0100, Loss: 26898.123047, 
Loss_residual_1: 9.263165, Loss_residual_2: 5.389154, Loss_residual_3: 4.696953, 
Loss_theta_1: 939.057129, Loss_theta_2

KeyboardInterrupt: 